# SPARK WORDCOUNT

Read Shakespeare text from HDFS. We can read the text in HDFS into a new variable called lines:

The SparkContext, sc, is the main entry point for accessing Spark in Python. The textFile() method reads the file into a Resilient Distributed Dataset (RDD) with each line in the file being an element in the RDD collection. The URL hdfs:/user/cloudera/words.txt specifies the location of the file in HDFS.

In [1]:
lines = sc.textFile("hdfs:/user/cloudera/words.txt")

We can verify the file was successfully loaded by calling the count() method, which prints the number of elements in the RDD:

In [2]:
lines.count()

124456

Split each line into words. Next, we will split each line into a set of words. To split each line into words and store them in an RDD called words.

In [5]:
words = lines.flatMap(lambda line : line.split(" "))

The flatMap() method iterates over every line in the RDD, and lambda line : line.split(" ") is executed on each line. The lambda notation is an anonymous function in Python, i.e., a function defined without using a name. In this case, the anonymous function takes a single argument, line, and calls split(" ") which splits the line into an array words.

Assign initial count value to each word. Next, we will create tuples for each word with an initial count of 1

In [6]:
tuples = words.map(lambda word : (word, 1))

The map() method iterates over every word in the words RDD, and the lambda expression creates a tuple with the word and a value of 1.

Note that in the previous step we used flatMap, but here we used map. In this step, we want to create a tuple for every word, i.e., we have a one-to-one mapping between the input words and output tuples. In the previous step, we wanted to split each line into a set of words, i.e., there is a one-to-many mapping between input lines and output words. In general, use map when the number of inputs to number of outputs is one-to-one, and flatMap for one-to-many (or one-to-none).

Sum all word count values. We can sum all the counts in the tuples for each word into a new RDD counts

In [7]:
counts = tuples.reduceByKey(lambda a, b: (a+b))

The reduceByKey() method calls the lambda expression for all the tuples with the same word. The lambda expression has two arguments, a and b, which are the count values in two tuples.

Write word counts to text file in HDFS. We can werite the counts RDD to HDFS:

In [9]:
counts.coalesce(1).saveAsTextFile("hdfs:/user/cloudera/wordcount/outputDir")

The coalesce() method combines all the RDD partitions into a single partition since we want a single output file, and saveAsTextFile() writes the RDD to the specified location.

View results. We can view the results by first copying the file from HDFS to the local file system and then running more:

hadoop fs -copyToLocal wordcount/outputDir/part-00000 count.txt
more count.txt